In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
persist_directory = 'docs/chroma'

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

C:\Users\busra\AppData\Local\Temp\ipykernel_17956\1787148411.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
C:\Users\busra\AppData\Local\Temp\ipykernel_17956\1787148411.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [4]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [5]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [6]:
question = 'Tell me about all-white mushrooms with large fruiting bodies'

In [7]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [8]:
smalldb.max_marginal_relevance_search(question, k=2, fetch_k=3)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [9]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [10]:
docs_ss[0].page_content[:100]

'that we saw earlier is known as a parametric learning algorithm, because\nit has a ﬁxed, ﬁnite number'

In [11]:
docs_ss[1].page_content[:100]

'that we saw earlier is known as a parametric learning algorithm, because\nit has a ﬁxed, ﬁnite number'

In [12]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
docs_mmr[0].page_content[:100]

'that we saw earlier is known as a parametric learning algorithm, because\nit has a ﬁxed, ﬁnite number'

In [13]:
docs_mmr[1].page_content[:100]

'9\nWe now state without proof some facts of matrix derivatives (we wo n’t\nneed some of these until la'

In [14]:
question = "what did they say about regression in the third lecture?"

In [15]:
docs = vectordb.similarity_search(
    question, k=3, 
    filter={"source":"C1_W3.pdf"}
)

In [16]:
for d in docs:
    print(d.metadata)

In [17]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [18]:
metadata_field_info = [
    AttributeInfo(
        name='source',
        description="The lecture the chunk is from, should be one of `C1_W1.pdf`, `C1_W2.pdf`, or `dC1_W3.pdf`",
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='The page from the lecture',
        type='integer'
    ),
]

In [19]:
import lark

In [20]:
from langchain.llms import OpenAI
document_content_description = "Lecture notes"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

C:\Users\busra\AppData\Local\Temp\ipykernel_17956\1028350957.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)


In [21]:
question = "what did they say about regression in the third lecture?"

In [25]:
docs = retriever.get_relevant_documents(question)

In [27]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [28]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [29]:
llm = OpenAI(model='gpt-3.5-turbo-instruct')
compressor = LLMChainExtractor.from_llm(llm)

In [30]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [32]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

- to make predictions using locally weighted linear regression, we need to keep the entire training set around.
- The term “non-parametric” (roughly) refers to the fact that the amount of stuff we need to keep in order to represent the hypothesis h grows linearly with the size of the training set.
----------------------------------------------------------------------------------------------------
Document 2:

None of the context is relevant.
----------------------------------------------------------------------------------------------------
Document 3:

- "In contrast, to make predictions using locally weighted linear regression, we need to keep the entire training set around."
- "The term “non-parametric” (roughly) refers to the fact that the amount of stuff we need to keep in order to represent the hypothesis h grows linearly with the size of the training set."


In [33]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [34]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

In cont rast, to make predictions using locally weighted linear regression, we need to k eep the entire training set around. The term “non-parametric” (roug hly) refers to the fact that the amount of stuﬀ we need to keep in order to rep resent the hypothesis h grows linearly with the size of the training set.
----------------------------------------------------------------------------------------------------
Document 2:

Equation (1) above states that the ( i, j) entry of this matrix will be given by
the ( i, j)-entry of BT , or equivalently, by Bji .
The proofs of Equations (1-3) are reasonably simple, and are left a s an
exercise to the reader.
----------------------------------------------------------------------------------------------------
Document 3:

The perceptron was argued to be a rough model for how individual neurons in the brain work.
----------------------------------------------------------------------------------------------------
Document 4:

"In class, w